# CRC Tutorial 

### iMAP workflow

In this tutorial, we will show the entire iMAP pipeline using the 'CRC' dataset to input the whole transcriptome(See <font color='red'>main text</font>). And we will show in detail how to do proper data preprocessing based on your own data set, without using the data preprocessing methods that provided by iMAP.

The total workfow includes: 
<ol>
    <li>Loading and preprocessing data;</li>
    <li>Running the main iMAP two-stage batch effect removal procedure;</li>
</ol>

In [ ]:
#### IMPORT LIBRARY ####
import scanpy as sc
import numpy as np
import imap.imap as imap
import imap.utils as utils

#### 1. Loading and preprocessing data

The 'CRC' dataset is already stored at <a href=''><font color='blue'>'../data/crc_10x.loom'</font></a> and <a href=''><font color='blue'>'../data/crc_smart.loom'</font></a>. We use the scanpy API to read the file and store as an 'AnnData' object. Here we removed epithelial, ribroblas and malignant cells in the Smart-seq2 sub-dataset and use single-cell transcriptomes of over 50,000 immune cells from 18 CRC patients to apply iMAP on the study of immune microenvironment.

In [ ]:
adata1 = sc.read_loom('../data/crc_10x.loom',sparse=False)
adata2 = sc.read_loom('../data/crc_smart.loom',sparse=False)
adata2 = adata2[[True if item not in ['Epithelial cell', 'Fibroblast', 'Malignant cell']  else False for item in adata2.obs['Global_Cluster']]]

**Preprocessing:** Since the Smart-seq2 data is of better quality, we aligned the 10X data to the Smart-seq2 data. Here we used highly variable genes data to do the Stage I and calculate rwMNN pairs. The whole transcriptome were used to do the Stage II. Genes that exist only on one platform were removed, and the number of remaining genes is 12667. We recorded there highly variable genes set of 10X sub-dataset, Smart-seq2 sub-dataset and the whole dataset, respectly. Then we used union of the three as Stage I's input, which contains 3892 highly variable genes.

In [ ]:
sc.pp.filter_cells(adata1, min_genes=200)
sc.pp.filter_genes(adata1, min_cells=3)
sc.pp.normalize_per_cell(adata1, counts_per_cell_after=1e4)
sc.pp.log1p(adata1)
sc.pp.highly_variable_genes(adata1, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes=None)

sc.pp.filter_cells(adata2, min_genes=200)
sc.pp.filter_genes(adata2, min_cells=3)
sc.pp.normalize_per_cell(adata2, counts_per_cell_after=1e4)
sc.pp.log1p(adata2)
sc.pp.highly_variable_genes(adata2, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes=None)

hv1_var_names = adata1.var_names[adata1.var['highly_variable']]
hv2_var_names = adata2.var_names[adata2.var['highly_variable']]

AnnData object stores annotation of 'Platform' and 'Sample' observations, which records sequencing platform and patient ID. We integrated these two types of information as the 'batch' annotation.

In [ ]:
cell_batch = []
for platform, sample in zip(adata1.obs['Platform'], adata1.obs['Sample']):
    cell_batch.append(','.join([platform, sample]))
for platform, sample in zip(adata2.obs['Platform'], adata2.obs['Sample']):
    cell_batch.append(','.join([platform, sample]))

In [ ]:
adata = adata1.concatenate(adata2)
adata.obs['batch'] = cell_batch
celltype1 = adata.obs['Global_Cluster']
celltype2 = adata.obs['Sub_Cluster']
print(adata)

AnnData object with n_obs × n_vars = 53018 × 12667 
    
obs: 'ClusterID', 'ClusterName', 'Global_Cluster', 'Platform', 'Sample', 'Sub_Cluster', 'Sub_ClusterID', 'Tissue', 'batch', 'n_counts', 'n_genes', 'P_batch'

var: 'Selected-0', 'vst_mean-0', 'vst_variable-0', 'vst_variance-0', 'vst_variance_expected-0', 'vst_variance_standardized-0', 'n_cells-0', 'highly_variable-0', 'means-0', 'dispersions-0', 'dispersions_norm-0', 'Selected-1', 'vst_mean-1', 'vst_variable-1', 'vst_variance-1', 'vst_variance_expected-1', 'vst_variance_standardized-1', 'n_cells-1', 'highly_variable-1', 'means-1', 'dispersions-1', 'dispersions_norm-1', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

layers: 'norm_data', 'scale_data'

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes=None)
hv_var_names = adata.var_names[adata.var['highly_variable']]
high_var_names = set()
high_var_names.update(list(hv_var_names))
high_var_names.update(list(hv1_var_names))
high_var_names.update(list(hv2_var_names))

adata_hv = adata[:,[True if item in high_var_names else False for item in adata.var_names]]

Because the data variation between different patients on the Smart-seq2 platform is small, the variation between patients on the 10X platform is significant. We treated all of Smart-seq2 patients' data as one batch.

In [ ]:
adata_hv.obs['P_batch'] = np.array([item if 'Smart-seq2,' not in item else 'Smart-seq2' for item in adata_hv.obs['batch']])
adata.obs['P_batch'] = adata_hv.obs['P_batch']
print(adata_hv)

AnnData object with n_obs × n_vars = 53018 × 3892 

obs: 'ClusterID', 'ClusterName', 'Global_Cluster', 'Platform', 'Sample', 'Sub_Cluster', 'Sub_ClusterID', 'Tissue', 'batch', 'n_counts', 'n_genes', 'P_batch'

var: 'Selected-0', 'vst_mean-0', 'vst_variable-0', 'vst_variance-0', 'vst_variance_expected-0', 'vst_variance_standardized-0', 'n_cells-0', 'highly_variable-0', 'means-0', 'dispersions-0', 'dispersions_norm-0', 'Selected-1', 'vst_mean-1', 'vst_variable-1', 'vst_variance-1', 'vst_variance_expected-1', 'vst_variance_standardized-1', 'n_cells-1', 'highly_variable-1', 'means-1', 'dispersions-1', 'dispersions_norm-1', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

layers: 'norm_data', 'scale_data'

#### 2. Batch effect removal by iMAP

See <font color='red'>Cell Line Tutorial</font> for details on the usage of `iMAP_fast` and `integrate_data`.

**Stage I:** This processure is the same as that in Cell Line Tutorial, except that the learning rate is adjusted to 0.001.

**Stage II:** For the sake of simplicity, instead of changing the model structure (the number of nodes in neural network) to accommodate the increase in the number of genes, we split the data into several pieces of sub-data of appropriate length, which were calculated separately and the final results were integrated together. Here we set `inc` to True to align better for the multi-batch CRC datasets.

The results are saved in './crc_data/'

In [ ]:
### Stage I
EC, ec_data = imap.iMAP_fast(adata_hv, key="P_batch", n_epochs=200, lr=0.001)

### Stage II
final_results = []
rindex = np.random.permutation(np.arange(len(adata.var_names)))
for i,(start, end) in enumerate([(0, 1809), (1809, 3618), (3618, 5427), (5427, 7236), (7236, 9045), (9045, 10854), (10854, 12667)]):
    output_results = utils.integrate_data(adata[:,rindex[start:end]], ec_data, key='P_batch', inc=True)
    final_results.append(output_results)

output_results = final_results[0]
for item in final_results[1:]:
    output_results = np.c_[output_results, item]
    
### Save Data
simple_adata = sc.AnnData(output_results)
simple_adata.var_names = adata.var_names[rindex]
simple_adata.obs_names = adata.obs_names
for key in adata.obs_keys():
    simple_adata.obs[key] = adata.obs[key]
simple_adata.write_loom(f'./crc_data/crc_immune_full_results.loom')